#### this part for cloud storage

In [ ]:
# !pip install --upgrade google-cloud-storage

In [ ]:
#save the location to credentials in path

In [ ]:
! source ~/.bash_profile

In [1]:
! echo $GOOGLE_APPLICATION_CREDENTIALS

/Users/amirulislam/projects/DBS_related/common_projects/poc_speech_transcribe/resources


In [ ]:
# Imports the Google Cloud client library
from google.cloud import storage



In [ ]:
# Instantiates a client
storage_client = storage.Client()



In [ ]:
# The name for the new bucket
bucket_name = 'my-xtra-cool-bucket'

# Creates the new bucket
bucket = storage_client.create_bucket(bucket_name)




In [ ]:
print('Bucket {} created.'.format(bucket.name))


In [ ]:
def upload_blob(bucket_name, source_file_name, destination_blob_name):
    """Uploads a file to the bucket."""
    storage_client = storage.Client()
    bucket = storage_client.get_bucket(bucket_name)
    blob = bucket.blob(destination_blob_name)

    blob.upload_from_filename(source_file_name)

    print('File {} uploaded to {}.'.format(
        source_file_name,
        destination_blob_name))

In [ ]:
bucket_name="my-xtra-cool-bucket"
source_file_name="/Users/amirulislam/projects/DBS_related/poc_speech_transcribe/sample_data/Untitled.ipynb"
destination_blob_name="sample_data"

In [ ]:
upload_blob(bucket_name,source_file_name,destination_blob_name)

In [ ]:
def download_blob(bucket_name, source_blob_name, destination_file_name):
    """Downloads a blob from the bucket."""
    storage_client = storage.Client()
    bucket = storage_client.get_bucket(bucket_name)
    blob = bucket.blob(source_blob_name)

    blob.download_to_filename(destination_file_name)

    print('Blob {} downloaded to {}.'.format(
        source_blob_name,
        destination_file_name))

In [ ]:
bucket_name="my-xtra-cool-bucket"
source_blob_name="sample_data"
destination_file_name="/Users/amirulislam/projects/DBS_related/poc_speech_transcribe/sample_data/downloads/sample_d.txt"

In [ ]:
download_blob(bucket_name,source_blob_name,destination_file_name)

#### Now going for speech to text

In [3]:
# ! pip install --upgrade google-cloud-speech

In [1]:
import io
import os

# Imports the Google Cloud client library
from google.cloud import speech
from google.cloud.speech import enums
from google.cloud.speech import types



In [13]:
__file__="../"

# Instantiates a client
client = speech.SpeechClient()

# The name of the audio file to transcribe
file_name = os.path.join(
    os.path.dirname(__file__),
    'resources',
    'audio.raw')
print(file_name)

# Loads the audio into memory
with io.open(file_name, 'rb') as audio_file:
    content = audio_file.read()
    audio = types.RecognitionAudio(content=content)
    


config = types.RecognitionConfig(
    encoding=enums.RecognitionConfig.AudioEncoding.FLAC,
    sample_rate_hertz=44100,
    language_code='en-US')

# Detects speech in the audio file
response = client.recognize(config, audio)

print("Result is ",response)

for result in response.results:
    print('Transcript: {}'.format(result.alternatives[0].transcript))

../resources/audio.raw
Result is  results {
  alternatives {
    transcript: "hello how are you what is your name"
    confidence: 0.9876290559768677
  }
}

Transcript: hello how are you what is your name


In [15]:
!afinfo "../resources/audio.raw"

File:           ../resources/audio.raw
File type ID:   flac
Num Tracks:     1
----
Data format:     1 ch,  44100 Hz, 'flac' (0x00000001) from 16-bit source, 4608 frames/packet
Channel layout: Mono
estimated duration: 3.448163 sec
audio bytes: 67562
audio packets: 33
bit rate: 156748 bits per second
packet size upper bound: 3177
maximum packet size: 3177
audio data file offset: 42
optimized
audio 0 valid frames + 0 priming + 0 remainder = 0
source bit depth: I16
----


#### speech to text for remote file

In [4]:
def transcribe_gcs(gcs_uri):
    """Transcribes the audio file specified by the gcs_uri."""
    from google.cloud import speech
    from google.cloud.speech import enums
    from google.cloud.speech import types
    client = speech.SpeechClient()

    audio = types.RecognitionAudio(uri=gcs_uri)
    config = types.RecognitionConfig(
        encoding=enums.RecognitionConfig.AudioEncoding.FLAC,
        sample_rate_hertz=44100,
        language_code='en-US')

    response = client.recognize(config, audio)
    # Each result is for a consecutive portion of the audio. Iterate through
    # them to get the transcripts for the entire audio file.
    for result in response.results:
        # The first alternative is the most likely one for this portion.
        print(u'Transcript: {}'.format(result.alternatives[0].transcript))

In [5]:

gcs_uri="gs://my-xtra-cool-bucket/sample_data"

In [6]:
transcribe_gcs(gcs_uri)

Transcript: hello how are you what is your name
